In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class ParticleSwarm():
    def __init__(self,  x_domain, v_max, cost_f, dimension, swarmsize, swarm=None, obj=None,   
                alpha = 0.72984, #0.72, 0.3925
                beta = 2.05, #1.5, 2.5586, 2.05
                gamma = 2.05): #1.5, 1.3358, 2.05
        self.dimension = dimension
        self.swarmsize = swarmsize
        self.cost_f = cost_f
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        if x_domain == np.inf:
            self.x_domain = 10
        else:    
            self.x_domain = x_domain
        self.v_max = v_max
        if swarm is None:
            # np.random.seed(63)
            self.swarm = np.random.uniform(-self.x_domain, self.x_domain, (self.dimension, self.swarmsize))*0.1
        else:
            self.swarm = swarm
        self.bestLocal  = self.swarm.copy() # лучшие положения частиц
        self.bestGlobal = np.random.uniform(-self.x_domain, self.x_domain, (self.dimension, 1))*0.1
        self.velocity = np.random.uniform( -self.x_domain, self.x_domain, (self.dimension, self.swarmsize)) * 0.1

    def optimum_all(self):
        f_best = self.cost_f(self.bestLocal[0], self.bestLocal[1])
        b = f_best >= self.cost_f(self.swarm[0], self.swarm[1]) 
        self.bestLocal[:, b] = self.swarm[:, b] #из каждой строки "bestLocal" и "swarm" формируем массив с индексами из "b"
        
        k = np.argmin(f_best)
        self.bestGlobal[0], self.bestGlobal[1] = self.bestLocal[0][k], self.bestLocal[1][k]
        
    def velocity_f(self):
        self.optimum_all()
        r1, r2 = np.random.rand(2)
        self.velocity = self.alpha * self.velocity + self.beta * r1 * (self.bestLocal - self.swarm) + self.gamma * r2 * (self.bestGlobal - self.swarm)
        self.velocity = np.minimum(self.velocity,  self.v_max)
        self.velocity = np.maximum(self.velocity, -self.v_max)


    def rate_particles(self):
        self.velocity_f()
        self.swarm = self.swarm + self.velocity #перемещение частицы
        self.swarm = np.minimum(self.swarm,  self.x_domain)
        self.swarm = np.maximum(self.swarm, -self.x_domain)

In [ ]:
class Jaya():
    def __init__(self, domain, cost_f, dimension, populationsize, population=None): 
        self.dimension = dimension
        self.populationsize = populationsize
        if domain == np.inf:
            self.domain = 10
        else:    
            self.domain = domain
        if population is None:
            self.population = np.random.uniform(-self.domain, self.domain, (self.dimension, self.populationsize))*0.1
        else:
            self.population = population
        self.population_updt  = self.population.copy() 
        self.cost_f = cost_f
        self.best = np.random.uniform(-self.domain, self.domain, (self.dimension, 1))*0.1 
        self.worst = np.random.uniform(-self.domain, self.domain, (self.dimension, 1))*0.1
        
    def best_worst(self):
        f_value = self.cost_f(self.population[0], self.population[1])

        k = np.argmin(f_value)
        self.best = self.population.T[k].reshape(-1,1)

        q = np.argmax(f_value)
        self.worst = self.population.T[q].reshape(-1,1)

    def update(self):
        self.best_worst()
        r1, r2 = np.random.rand(2)
        self.population_updt = self.population + r1 * (self.best - abs(self.population)) - r2 * (self.worst - abs(self.population))

        b = self.cost_f(self.population_updt[0], self.population_updt[1]) <= self.cost_f(self.population[0], self.population[1]) 
        self.population.T[b] = self.population_updt.T[b] 

        self.population = np.minimum(self.population,  self.domain)
        self.population = np.maximum(self.population, -self.domain)  

In [ ]:
#functions for optimization
def f(x, y):
    #  return 20 + ((x**2 - 10 * np.cos(2 * np.pi * x)) + (y**2 - 10 * np.cos(2 * np.pi * y))) # Rastrigin -5.12, 5.12  min f(0, 0) = 0
    # return x**2 + y**2
    #  return sum(abs(x)) #f(0,0)=0 [-10, 10]
    # return abs(x + y) 
    # return np.sin(x1 - mt.pi / 2) + np.cos(x2)
    # return 	-20.0 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) + np.e + 20 #Экли -5;5  min f(0, 0) = 0
    # return - np.abs(np.sin(x) * np.cos(y) * np.exp(np.abs(1 - np.sqrt(x ** 2 + y ** 2) / np.pi))) #Хольдер -10 <= x, y <= 10 min f(8.05502, 9.66459) = -19.2085
    # return np.sin(x + y) + (x - y)**2 - 1.5 * x + 2.5 * y + 1 #Функция МакКормика -3 < y < 4
    # return (x**2 + y - 11)**2 + (x + y**2 - 7)**2 #Himmelblau -5 <= x, y <= 5 f(3.0,2.0)=0.0\\f(-2.805118,3.131312)=0.0\\f(-3.779310,-3.283186)=0.0\\f(3.584428,-1.848126)=0.0
    # return - np.cos(x) * np.cos(y) * np.exp( - ((x - np.pi)**2 + (y - np.pi)**2)) # Izom -100 <= x, y <= 100  f(pi ,pi )=-1
    return 0.26 * (x**2 + y**2) - 0.48 * x * y # Матьяс -10 <= x, y <= 10 min f(0, 0) = 0
    # return (x + 2 * y - 7)**2 + (2 * x + y - 5)**2 # Бут min f(1,3) = 0 -10 <= x, y <= 10
    # return (np.sin(3 * np.pi * x))**2 + (x - 1)**2 * (1 + (np.sin(3 * np.pi * y))**2) + (y - 1)**2 * ((1 + (np.sin(2 * np.pi * y))**2)) # Леви, f(1,1) = 0, -10 <= x, y <= 10
    # return 0.5 + ((np.cos(np.sin(abs(x**2 - y**2))))**2 - 0.5) / (1 + 0.001 * (x**2 + y**2))**2 #-100;100 Шаффер №4 f(0,1.25313)=0.292579
    # return -0.0001 * (np.abs(np.sin(x) * np.sin(y) * np.exp(np.abs(100 - np.sqrt(x**2 + y**2) / np.pi))) + 1)**0.1 # "крест на подносе", f(1.34941, 1.34941) = -2.06261, -10 <= x, y <= 10
    # return  -(y + 47) * np.sin(np.sqrt(np.abs(x / 2 + (y + 47)))) - x * np.sin(np.sqrt(np.abs(x - (y + 47)))) # "подставка для яиц" Egg holder function, f(512, 404.2319) = -959.6407, -512 <= x, y <= 512
    # return 1 + (x**2 + y**2) / 4000 - np.cos(x) * np.cos(y/np.sqrt(2)) # функция Гриванка f(0,0) = 0,  -600 <= x,y <= 600
    # return  -(x * np.sin(np.sqrt(np.abs(x))) + y * np.sin(np.sqrt(np.abs(y))))
    # return 4 * x**2 - 2.1 * x**4 + 1/3 * x**6 + x * y - 4 * y**2 + 4 * y**4 # f(0.08983, -0.7126) = -1.0316285
    # return (x-1)**2 + 100 * (y - x**2)**2 #f(1,1)=0 Розенброк
 

In [ ]:
# algorithm parameters 
#min_f = 0.292579#-2.06261 -959.6407 0.292579  -19.2085
min_f = 0
dom = 10
num_iter = 200
num_sim = 50
#######################


In [ ]:
# PSO
all_err_pso = []
for k in range(num_sim):
    mean_list_pso = []
    pso_one = ParticleSwarm(x_domain=dom, v_max=np.inf, dimension=2, swarmsize=10, cost_f=f)
    for i in range(num_iter):
        pso_one.rate_particles()
        mean_list_pso.append(np.abs(f(pso_one.bestGlobal[0][0], pso_one.bestGlobal[1][0]) - min_f))
    all_err_pso.append(mean_list_pso)


In [ ]:
#Jaya
all_err_jy = []
for k in range(num_sim):
    mean_list_jy = []
    jy_one = Jaya(domain=dom, cost_f=f, dimension=2, populationsize=10)
    jy_one.best_worst()
    for i in range(num_iter):
        jy_one.update()
        minimum = np.abs((f(jy_one.best[0][0], jy_one.best[1][0]) - min_f))
        mean_list_jy.append(minimum)
    all_err_jy.append(mean_list_jy)

In [ ]:
#PSO-Jaya
all_err_pso_jy = []
for k in range(num_sim):
    pso = ParticleSwarm(x_domain=dom, v_max=np.inf, dimension=2, swarmsize=10, cost_f=f)
    mean_list_pso_jaya = [np.abs(f(pso.bestGlobal[0][0], pso.bestGlobal[1][0]) - min_f)]
    for i in range(num_iter):
        pso.rate_particles()
        if np.abs(f(pso.bestGlobal[0][0], pso.bestGlobal[1][0]) - min_f) >= mean_list_pso_jaya[-1]:
            jy = Jaya(domain=pso.x_domain, cost_f=pso.cost_f, population=pso.swarm, dimension=pso.dimension, populationsize=pso.swarmsize)
            for j in range(10):
                jy.update()
            pso.swarm = jy.population
        mean_list_pso_jaya.append(np.abs(f(pso.bestGlobal[0][0], pso.bestGlobal[1][0]) - min_f))
    all_err_pso_jy.append(mean_list_pso_jaya)


In [ ]:
def mean_all_simul(all_err):
    all_err = np.asarray(all_err)
    mean_list = []
    for el in all_err.T:
        mean_list.append(el.mean())
    return np.asarray(mean_list)


In [ ]:
def print_statistical_analysis(all_best_scores):
    all_best_scores = np.asarray(all_best_scores)
    print("mean:", np.mean(all_best_scores.T[-1]))
    print("median:", np.median(all_best_scores.T[-1]))
    print("std:", np.std(all_best_scores.T[-1]))
    print("min", np.min(all_best_scores.T[-1]))

In [ ]:
print("--pso--")
print_statistical_analysis(all_err_pso)
print("--jaya--")
print_statistical_analysis(all_err_jy)
print("--pso_jaya--")
print_statistical_analysis(all_err_pso_jy)

In [ ]:
import matplotlib.pyplot as plt
mean_list_pso = mean_all_simul(all_err_pso)
mean_list_jy = mean_all_simul(all_err_jy)
mean_list_pso_jaya = mean_all_simul(all_err_pso_jy)
n = np.arange(mean_list_pso.shape[0])

plt.plot(n, mean_list_pso, "--r", label="pso")
plt.plot(n, mean_list_jy, 'b', label="jaya")
plt.plot(n, mean_list_pso_jaya[1:201], ':g',  label="pso_jaya")

plt.xlabel("iteration")
plt.ylabel("error")
plt.legend()
plt.grid('True')
plt.semilogy()


In [ ]:
all_err_pso = np.asarray(all_err_pso)
all_err_jy = np.asarray(all_err_jy)
all_err_pso_jy = np.asarray(all_err_pso_jy)

df = pd.DataFrame({'PSO': all_err_pso.T[-1], 'Jaya': all_err_jy.T[-1], 'pso_jaya': all_err_pso_jy.T[-1]})
#create boxplot by group
plt.boxplot(df, labels=['pso', 'jaya', 'pso-jaya'])
plt.xlabel("algorithm")
plt.ylabel("error")
plt.semilogy()
